In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from tqdm import tqdm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models

import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

batch_size=32
num_epoches=5

In [ ]:
!tar -zxvf ../input/cifar10-python/cifar-10-python.tar.gz
os.listdir('.')

In [ ]:

trans = transforms.Compose([transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset=torchvision.datasets.CIFAR10(root='.', train=True, download=False, transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)

trans_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
testset = torchvision.datasets.CIFAR10(root='.', train=False, download=False, transform=trans_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=2*batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# image, label = trainset[0]
# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
# imshow(image)
# print(classes[label])
#load succefully

set up the general Resnet
<br/>Note:
1. nn.Conv2d's parameters are input's channels, output channels. Don't care the size of the image


In [ ]:
class bottleneck(nn.Module):
    def __init__(self,in_planes, planes,stride=1):
        super(bottleneck,self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                                stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, 4*planes,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(4*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != 4*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, 4*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(4*planes)
            )
            pass
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out 

In [ ]:
class Resnet50(nn.Module):
    def __init__(self):
        super(Resnet50,self).__init__()
        self.current_input = 64
        self.conv1=nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
        self.BN1=nn.BatchNorm2d(64)
        # generate the mid-layers the pattern is:
        #(1*1[64]->3*3[64]->1*1[256])*3
        #(1*1[128]->3*3[128]->1*1[512])*4
        #(1*1[256]->3*3[256]->1*1[1024])*6
        ##(1*1[512]->3*3[512]->1*1[2048])*3
        self.layer_1=self._generate_layers(64,3,stride=1)# (inputs,number)
        self.layer_2=self._generate_layers(128,4,stride=2)
        self.layer_3=self._generate_layers(256,6,stride=2)
        self.layer_4=self._generate_layers(512,3,stride=2)
        
        self.linear = nn.Linear(2048,10)
        pass
    
    def _generate_layers(self,des_output,numbers,stride):
        strides = [stride] + [1]*(numbers-1)
        layers=[]
        for stride in strides:
            layers.append(bottleneck(self.current_input,des_output,stride))
            self.current_input = des_output*4
            pass
        return nn.Sequential(*layers)
    
    def forward(self,x):
        out = F.relu(self.BN1(self.conv1(x)))
        out = self.layer_1(out)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = F.avg_pool2d(out,4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
    
net=Resnet50()
    
        
        
        

Training

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
losses=[]
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(trainloader)
    i = 0

    for data in pbar:
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
            
        if i%100==0:
            losses.append(loss.data.numpy())
            print(losses)
            pass

        pbar.set_description("Processing epoch {:d} minibatch {:d} train loss {:.3f}".format(epoch,\
                                                                i+1, running_loss/(i+1)))
        i += 1      
print(losses)

print('Finished Training')





In [ ]:
plt.plot(np.arange(len(losses)),losses)


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for i,data in enumerate(testloader, 0):
        net.eval()
        img,labels = data
        outputs=net(img)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted==labels).sum()
        if (i+1)%10==0:
             print("the correct is %.3f%%"%(100*correct/total))
     
            
   